In [1]:
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

In [8]:
import pandas as pd
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')
X_train

,latitude,longitude,land_use_label,distance_to_waterbody,distance_to_open_space,subzone,planning_area,region,elevation,temp_2024_04_07_min,...,built-up,bare / sparse vegetation,snow and ice,permanent water bodies,herbaceous wetland,mangroves,moss and lichen,min_ndvi,mean_ndvi,max_ndvi
0,1.327345,103.776261,ROAD,0.005491,0.000305,HOLLAND ROAD,BUKIT TIMAH,CENTRAL REGION,34,28.880736,...,128,1,0,1,0,0,0,0.1176063463,0.2107233339,0.3355351585
1,1.362310,103.885041,RESIDENTIAL,0.002163,0.002288,KOVAN,HOUGANG,NORTH-EAST REGION,14,33.603571,...,183,1,0,0,0,0,0,0.06873453002,0.1237388913,0.1772913102
2,1.304792,103.740678,BUSINESS 2,0.001660,0.001437,PENJURU CRESCENT,JURONG EAST,WEST REGION,10,28.880736,...,251,8,0,33,0,0,0,0.03399855502,0.07334574643,0.1149060753
3,1.432131,103.793028,ROAD,0.002688,0.002472,WOODLANDS SOUTH,WOODLANDS,NORTH REGION,32,30.168782,...,-,-,-,-,-,-,-,-,-,-
4,1.303530,103.820861,CIVIC & COMMUNITY INSTITUTION,0.011124,0.004127,RIDOUT,TANGLIN,CENTRAL REGION,17,30.168782,...,63,1,0,0,0,0,0,0.09017470784,0.2076336658,0.3255961435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,1.405039,103.911985,ROAD,0.000113,0.000136,WATERWAY EAST,PUNGGOL,NORTH-EAST REGION,8,28.451387,...,-,-,-,-,-,-,-,-,-,-
931,1.363962,103.736807,EDUCATIONAL INSTITUTION,0.002075,0.001401,PARK,TENGAH,WEST REGION,13,28.022039,...,-,-,-,-,-,-,-,-,-,-
932,1.292290,103.629046,BUSINESS 2,0.004030,0.049930,TUAS VIEW,TUAS,WEST REGION,9,28.880736,...,-,-,-,-,-,-,-,-,-,-
933,1.335135,103.887696,BUSINESS 1 - WHITE,0.000849,0.001990,JOO SENG,TOA PAYOH,CENTRAL REGION,10,28.880736,...,92,1,0,0,0,0,0,0.05514517186,0.1176944237,0.1701961006


In [9]:
columns_to_drop = ["land_use_label", "subzone", "planning_area", "region"]
X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)
# X_train = X_train.replace('-', 0)
# X_test = X_test.replace('-', 0)
X_train = pd.concat([X_train, y_train], axis=1)
X_test = pd.concat([X_test, y_test], axis=1)
X_train = X_train[X_train["min_ndvi"] != '-']
X_test = X_test[X_test["min_ndvi"] != '-']
y_train = X_train['avg_temp']
X_train = X_train.drop(columns=['avg_temp'])
y_test = X_test['avg_temp']
X_test = X_test.drop(columns=['avg_temp'])
X_test

,latitude,longitude,distance_to_waterbody,distance_to_open_space,elevation,temp_2024_04_07_min,temp_2024_04_07_max,temp_2024_04_07_median,temp_2024_04_08_min,temp_2024_04_08_max,...,built-up,bare / sparse vegetation,snow and ice,permanent water bodies,herbaceous wetland,mangroves,moss and lichen,min_ndvi,mean_ndvi,max_ndvi
0,1.291346,103.855487,0.002468,0.000000,12,30.168782,31.886176,30.168782,28.022039,33.603571,...,60,7,0,8,0,0,0,0.05633863817,0.1140787686,0.1792005627
2,1.332069,103.701299,0.002303,0.003304,7,28.880736,34.032919,33.388896,28.451387,34.462268,...,171,2,0,1,0,0,0,0.06673310083,0.1200177176,0.1936944068
3,1.342354,103.711989,0.004304,0.000705,15,30.168782,30.168782,30.168782,28.451387,33.603571,...,62,1,0,0,0,0,0,0.07350641175,0.1382490683,0.2319728431
5,1.364232,103.972470,0.005481,0.001296,25,28.451387,33.174222,30.168782,28.022039,34.032919,...,59,1,0,0,0,0,0,0.08484712292,0.1792649673,0.2799875709
7,1.337899,103.941388,0.000624,0.001216,14,28.880736,28.880736,28.880736,26.733993,30.168782,...,268,20,0,1,0,0,0,0.08510808138,0.1464489086,0.2171969832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,1.319596,103.982334,0.002208,0.000430,9,28.880736,28.880736,28.880736,27.592690,29.310085,...,283,26,0,25,0,0,0,0.09192961094,0.1859358011,0.2627501899
229,1.337733,103.764978,0.006060,0.001170,44,30.168782,33.174222,31.886176,28.451387,34.032919,...,216,2,0,0,0,0,0,0.0944881155,0.1798681369,0.2836337334
230,1.325836,103.900057,0.001324,0.002603,14,28.880736,33.603571,33.174222,28.451387,34.032919,...,166,2,0,1,0,0,0,0.06275009389,0.1157940749,0.1702714913
232,1.318528,103.806201,0.004087,0.000264,23,28.880736,30.168782,30.168782,28.022039,34.032919,...,42,1,0,1,0,0,0,0.08377686003,0.1731302833,0.2627680605


In [10]:
# Get the data types of each column
column_types = X_train.dtypes

# Filter out columns with string data types
string_columns = column_types[column_types == 'object'].index.tolist()

# Print the columns with string values
print("Columns with string values:", string_columns)

Columns with string values: ['tree cover', 'shrubland', 'grassland', 'cropland', 'built-up', 'bare / sparse vegetation', 'snow and ice', 'permanent water bodies', 'herbaceous wetland', 'mangroves', 'moss and lichen', 'min_ndvi', 'mean_ndvi', 'max_ndvi']


In [11]:
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "warm_start": True,
    "oob_score": True,
    "random_state": 42
}

reg = RandomForestRegressor(**params)
reg.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, min_samples_split=5, n_estimators=500,
                      oob_score=True, random_state=42, warm_start=True)

In [12]:
y_pred = reg.predict(X_test)

In [13]:
mean_squared_error(y_test, y_pred, squared=False)

0.11292628176892788

In [14]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

0.9901750550306209

In [ ]:
y_pred

array([30.29107912, 27.89282643, 31.71327866, 30.69400387, 29.34766728,
       30.69734675, 29.16044525, 28.85906685, 28.72288254, 32.02229581,
       32.35430693, 28.87902562, 29.09431364, 31.62241166, 29.16044525,
       30.27753412, 32.166531  , 27.89554941, 30.64032502, 28.88475483,
       28.00079104, 30.28849783, 28.63878612, 30.05120481, 30.05017372,
       31.81105336, 28.88475483, 30.69734675, 30.0084047 , 29.62770148,
       28.20274282, 31.61651559, 29.09476093, 29.70676312, 28.3745219 ,
       28.85333764, 29.27464534, 28.72288254, 28.19825436, 30.69694378,
       29.48965757, 27.89554941, 30.4904749 , 29.61395072, 30.68955768,
       30.69440685, 29.20780808, 30.69694378, 31.98471211, 29.45906502,
       29.27286626, 32.02229581, 28.88475483, 31.74887287, 30.05153028,
       29.56059412, 29.73232381, 27.89282643, 28.72288254, 27.99560843,
       31.97278389, 28.88475483, 29.09431364, 29.16044525, 31.98269326,
       29.09431364, 28.72288254, 30.28283612, 30.63771742, 30.05